In [3]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
# natural language toolkit for textual data
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
df = pd.read_csv(r"C:\Users\HP\Downloads\archive\Sentiment.csv")
df = df[['text','sentiment']]

In [5]:
df.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [25]:
from sklearn.model_selection import train_test_split
df.shape

(1734, 2)

In [7]:
df.isnull().sum()

text         0
sentiment    0
dtype: int64

In [8]:
x=df['text'].values
y=df['sentiment'].values

In [9]:
print(x)

['RT @NancyLeeGrahn: How did everyone feel about the Climate Change question last night? Exactly. #GOPDebate'
 'Deer in the headlights RT @lizzwinstead: Ben Carson, may be the only brain surgeon who has performed a lobotomy on himself. #GOPDebate'
 'RT @CarolCNN: #DonaldTrump under fire for comments about women @PeterBeinart @SL_Schaeffer @IWF @@MyRkiger weigh in on #GOPdebate http://t.…'
 ...
 'RT @PuestoLoco: .@NewDay Cancel the primaries. Fox Party set up &amp; anointed their man Jeb Bush\n#GOPDebates #ObviousAsHell http://t.co/bgzYsy…'
 'GOP respects life....just not black ones. #GOPDebates'
 'RT @RWSurferGirl: Is it just me or does anyone else want to punch Chris Wallace in the face? #GOPDebate  #GOPDebates 🇺🇸']


In [10]:
print(y)

['Neutral' 'Negative' 'Negative' ... 'Negative' 'Negative' 'Negative']


In [11]:
df.head(10)

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
8,Deer in the headlights RT @lizzwinstead: Ben C...,Negative
16,RT @CarolCNN: #DonaldTrump under fire for comm...,Negative
24,RT @MrPooni: Fox News trying to convince us yo...,Negative
32,The First #GOPDebate: Social Media Reaction an...,Neutral
40,RT @mgell: Trump is a cross between the shrug ...,Negative
48,“@DanScavino: #GOPDebate w/ @realDonaldTrump d...,Positive
56,RT @Bipartisanism: The #GOPDebate response to ...,Negative
64,RT @TheJimHughes: If we want to stop sending f...,Negative
72,"RT @NaughtyBeyotch: RT @AllenWestRepub ""Dear @...",Negative


In [12]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)

In [13]:
print(x.shape,x_train.shape,x_test.shape)

(1734,) (1387,) (347,)


In [14]:
#converting the actual data to numerical data
vectorizer=TfidfVectorizer()
x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)

In [15]:
from pyswarm import pso
# Define the objective function for PSO
def objective_function(x):
    C = 10 ** x[0] 
    gamma = 10 ** x[1] 
    model=LogisticRegression(max_iter=1000)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    return -accuracy_score(y_test, y_pred)  # Negative since PSO minimizes

# Set up the PSO optimizer
num_dimensions = 2  
lb = [-1, -10]  
ub = [1, 10] 
x_opt, f_opt = pso(objective_function, lb, ub, swarmsize=20, maxiter=100)
# Convert the optimal solution to C and gamma values
C = 10 ** x_opt[0]
gamma = 10 ** x_opt[1]

Stopping search: maximum iterations reached --> 100


In [17]:
# Train the final model using the best hyperparameters
final_model = LogisticRegression()
final_model.fit(x_train, y_train)

#accuracy score on the training data
x_train_pred=final_model.predict(x_train)
train_data_acc=accuracy_score(y_train,x_train_pred)
print("Training Data Accuracy",train_data_acc)

Training Data Accuracy 0.7966834895457823


In [21]:
#accuracy score on the testing data
X_test_pred=final_model.predict(x_test)
test_data_acc=accuracy_score(y_test,X_test_pred)
print("Testing Data Accuracy",test_data_acc)

Testing Data Accuracy 0.6138328530259366


In [22]:
y_pred = final_model.predict(x_test)
final_accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the final model:", final_accuracy)

Accuracy of the final model: 0.6138328530259366


In [23]:
print(classification_report(y_test, X_test_pred))
print(y_test)

              precision    recall  f1-score   support

    Negative       0.65      0.91      0.76       207
     Neutral       0.37      0.21      0.27        77
    Positive       0.73      0.13      0.22        63

    accuracy                           0.61       347
   macro avg       0.58      0.42      0.41       347
weighted avg       0.60      0.61      0.55       347

['Neutral' 'Negative' 'Negative' 'Neutral' 'Neutral' 'Negative' 'Positive'
 'Neutral' 'Negative' 'Positive' 'Positive' 'Negative' 'Negative'
 'Negative' 'Negative' 'Negative' 'Positive' 'Negative' 'Negative'
 'Positive' 'Negative' 'Negative' 'Negative' 'Negative' 'Negative'
 'Negative' 'Negative' 'Positive' 'Negative' 'Neutral' 'Negative'
 'Negative' 'Negative' 'Negative' 'Negative' 'Negative' 'Positive'
 'Negative' 'Negative' 'Negative' 'Neutral' 'Negative' 'Neutral' 'Neutral'
 'Negative' 'Neutral' 'Negative' 'Negative' 'Negative' 'Neutral'
 'Negative' 'Negative' 'Neutral' 'Positive' 'Negative' 'Neutral' 'Neutr

In [24]:
positive_mask = (y_test =='Positive')
negative_mask = (y_test == 'Negative')
neutral_mask = (y_test == 'Neutral')

# Calculate accuracy for each sentiment category
accuracy_positive = accuracy_score(y_test[positive_mask], X_test_pred[positive_mask])
accuracy_negative = accuracy_score(y_test[negative_mask], X_test_pred[negative_mask])
accuracy_neutral = accuracy_score(y_test[neutral_mask], X_test_pred[neutral_mask])

print("Accuracy for Positive Sentiments:", accuracy_positive)
print("Accuracy for Negative Sentiments:", accuracy_negative)
print("Accuracy for Neutral Sentiments:", accuracy_neutral)

Accuracy for Positive Sentiments: 0.12698412698412698
Accuracy for Negative Sentiments: 0.9130434782608695
Accuracy for Neutral Sentiments: 0.2077922077922078
